# Fabric Tenant Settings Monitor

This notebook collects Microsoft Fabric tenant settings using the **Tenant Settings API** and ingests them into **Azure Log Analytics** for monitoring, alerting, and compliance reporting.

## Features
- Authenticate using Azure AD and retrieve tenant settings
- Identify preview features automatically
- Transform data for Log Analytics DCR-based ingestion
- Track configuration changes over time
- Generate alerts for new preview features

## Prerequisites
- **Fabric Administrator** role
- Azure Log Analytics workspace with DCR-based custom table
- Data Collection Endpoint (DCE) and Data Collection Rule (DCR)
- Environment variables configured (see `.env.example`)

In [ ]:
# Import required libraries
import os
import json
import requests
from datetime import datetime, timezone
from typing import Dict, List, Any
from azure.identity import DefaultAzureCredential
from azure.monitor.ingestion import LogsIngestionClient
from azure.core.exceptions import HttpResponseError

print("✓ Libraries imported successfully")

## Configuration

Load environment variables for Azure and Fabric configuration.

In [ ]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Azure Monitor configuration
DCE_ENDPOINT = os.getenv("AZURE_MONITOR_DCE_ENDPOINT")
DCR_IMMUTABLE_ID = os.getenv("AZURE_MONITOR_DCR_IMMUTABLE_ID")
STREAM_NAME = os.getenv("AZURE_MONITOR_STREAM_NAME", "Custom-FabricTenantSettings_CL")

# Fabric configuration
FABRIC_TENANT_ID = os.getenv("FABRIC_TENANT_ID") or os.getenv("AZURE_TENANT_ID")

# Validation
required_vars = {
    "DCE_ENDPOINT": DCE_ENDPOINT,
    "DCR_IMMUTABLE_ID": DCR_IMMUTABLE_ID,
    "FABRIC_TENANT_ID": FABRIC_TENANT_ID
}

missing_vars = [k for k, v in required_vars.items() if not v]
if missing_vars:
    raise ValueError(f"Missing required environment variables: {', '.join(missing_vars)}")

print("✓ Configuration loaded successfully")
print(f"  DCE Endpoint: {DCE_ENDPOINT[:50]}...")
print(f"  DCR Immutable ID: {DCR_IMMUTABLE_ID[:20]}...")
print(f"  Stream Name: {STREAM_NAME}")

## Authentication

Authenticate using **DefaultAzureCredential** which supports:
- Azure CLI authentication (for local development)
- Managed Identity (for production deployments)
- Environment variables (service principal)

In [ ]:
# Initialize Azure credential
credential = DefaultAzureCredential()

# Get access token for Fabric API
try:
    fabric_token = credential.get_token("https://api.fabric.microsoft.com/.default")
    print("✓ Successfully authenticated to Fabric API")
    print(f"  Token expires: {datetime.fromtimestamp(fabric_token.expires_on)}")
except Exception as e:
    print(f"✗ Authentication failed: {e}")
    raise

## Retrieve Tenant Settings

Call the **Fabric Tenant Settings API** to retrieve the complete tenant configuration.

In [ ]:
def get_tenant_settings(access_token: str) -> Dict[str, Any]:
    """
    Retrieve tenant settings from Fabric API.
    
    Args:
        access_token: Azure AD access token for Fabric API
        
    Returns:
        Dictionary containing tenant settings response
    """
    url = "https://api.fabric.microsoft.com/v1/admin/tenantsettings"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        if e.response.status_code in [401, 403]:
            raise PermissionError(
                "Access denied. Ensure you have Fabric Administrator role."
            ) from e
        raise
    except Exception as e:
        raise RuntimeError(f"Failed to retrieve tenant settings: {e}") from e

# Retrieve tenant settings
print("Retrieving tenant settings from Fabric API...")
tenant_settings_response = get_tenant_settings(fabric_token.token)
tenant_settings = tenant_settings_response.get("tenantSettings", [])

print(f"✓ Retrieved {len(tenant_settings)} tenant settings")

## Identify Preview Features

Analyze tenant settings to identify which ones are preview features based on naming patterns and metadata.

In [ ]:
def identify_preview_features(settings: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Identify preview features from tenant settings.
    
    Args:
        settings: List of tenant settings
        
    Returns:
        List of settings with preview feature metadata added
    """
    preview_keywords = [
        "preview", "experimental", "beta", "early access",
        "public preview", "private preview", "coming soon"
    ]
    
    enhanced_settings = []
    
    for setting in settings:
        title = setting.get("title", "").lower()
        description = setting.get("description", "").lower()
        
        # Check for preview indicators
        is_preview = False
        preview_indicators = []
        
        for keyword in preview_keywords:
            if keyword in title or keyword in description:
                is_preview = True
                preview_indicators.append(keyword)
        
        # Create enhanced setting object
        enhanced_setting = {
            "SettingName": setting.get("settingName", ""),
            "Title": setting.get("title", ""),
            "Enabled": setting.get("enabled", False),
            "CanSpecifySecurityGroups": setting.get("canSpecifySecurityGroups", False),
            "TenantSettingGroup": setting.get("tenantSettingGroup", ""),
            "IsPreviewFeature": is_preview,
            "PreviewIndicators": ", ".join(preview_indicators) if preview_indicators else None
        }
        
        enhanced_settings.append(enhanced_setting)
    
    return enhanced_settings

# Identify preview features
print("Analyzing settings for preview features...")
enhanced_settings = identify_preview_features(tenant_settings)

preview_count = sum(1 for s in enhanced_settings if s["IsPreviewFeature"])
enabled_previews = sum(1 for s in enhanced_settings if s["IsPreviewFeature"] and s["Enabled"])

print(f"✓ Analysis complete:")
print(f"  Total settings: {len(enhanced_settings)}")
print(f"  Preview features: {preview_count}")
print(f"  Enabled previews: {enabled_previews}")

## Transform for Log Analytics

Transform the data to match the Log Analytics DCR schema requirements.

In [ ]:
def transform_for_log_analytics(settings: List[Dict[str, Any]], tenant_id: str) -> List[Dict[str, Any]]:
    """
    Transform tenant settings for Log Analytics ingestion.
    
    Args:
        settings: List of enhanced tenant settings
        tenant_id: Fabric tenant ID
        
    Returns:
        List of records ready for Log Analytics ingestion
    """
    timestamp = datetime.now(timezone.utc).isoformat()
    
    records = []
    for setting in settings:
        record = {
            "TimeGenerated": timestamp,
            "TenantId": tenant_id,
            "SettingName": setting["SettingName"],
            "Title": setting["Title"],
            "Enabled": setting["Enabled"],
            "CanSpecifySecurityGroups": setting["CanSpecifySecurityGroups"],
            "TenantSettingGroup": setting["TenantSettingGroup"],
            "IsPreviewFeature": setting["IsPreviewFeature"],
            "PreviewIndicators": setting.get("PreviewIndicators") or "",
            "CollectionTimestamp": timestamp
        }
        records.append(record)
    
    return records

# Transform data
print("Transforming data for Log Analytics...")
log_records = transform_for_log_analytics(enhanced_settings, FABRIC_TENANT_ID)

print(f"✓ Transformed {len(log_records)} records")
print(f"  Sample record keys: {list(log_records[0].keys())}")

## Ingest to Log Analytics

Send the transformed data to Azure Log Analytics using the **Logs Ingestion API**.

In [ ]:
def ingest_to_log_analytics(
    records: List[Dict[str, Any]],
    dce_endpoint: str,
    dcr_id: str,
    stream_name: str,
    credential
) -> None:
    """
    Ingest records to Azure Log Analytics.
    
    Args:
        records: List of records to ingest
        dce_endpoint: Data Collection Endpoint URL
        dcr_id: Data Collection Rule immutable ID
        stream_name: Stream name defined in DCR
        credential: Azure credential object
    """
    client = LogsIngestionClient(endpoint=dce_endpoint, credential=credential, logging_enable=True)
    
    # Chunk records to stay under 1MB limit
    chunk_size = 500
    total_chunks = (len(records) + chunk_size - 1) // chunk_size
    
    print(f"Ingesting {len(records)} records in {total_chunks} chunk(s)...")
    
    for i in range(0, len(records), chunk_size):
        chunk = records[i:i + chunk_size]
        chunk_num = (i // chunk_size) + 1
        
        try:
            client.upload(
                rule_id=dcr_id,
                stream_name=stream_name,
                logs=chunk
            )
            print(f"  ✓ Chunk {chunk_num}/{total_chunks}: {len(chunk)} records ingested")
        except HttpResponseError as e:
            print(f"  ✗ Chunk {chunk_num}/{total_chunks} failed: {e}")
            raise

# Ingest data
try:
    print("\nIngesting data to Log Analytics...")
    ingest_to_log_analytics(
        records=log_records,
        dce_endpoint=DCE_ENDPOINT,
        dcr_id=DCR_IMMUTABLE_ID,
        stream_name=STREAM_NAME,
        credential=credential
    )
    print("✓ Ingestion completed successfully")
except Exception as e:
    print(f"✗ Ingestion failed: {e}")
    raise

## Summary & Preview Feature Report

Generate a summary report with focus on preview features.

In [ ]:
# Generate summary report
print("\n" + "="*60)
print("TENANT SETTINGS MONITORING SUMMARY")
print("="*60)

print(f"\nCollection Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Tenant ID: {FABRIC_TENANT_ID}")

print(f"\nSettings Overview:")
print(f"  Total Settings: {len(enhanced_settings)}")
print(f"  Enabled Settings: {sum(1 for s in enhanced_settings if s['Enabled'])}")
print(f"  Disabled Settings: {sum(1 for s in enhanced_settings if not s['Enabled'])}")

print(f"\nPreview Features:")
print(f"  Total Preview Features: {preview_count}")
print(f"  Enabled Previews: {enabled_previews}")
print(f"  Disabled Previews: {preview_count - enabled_previews}")

# List enabled preview features
enabled_preview_list = [
    s for s in enhanced_settings 
    if s["IsPreviewFeature"] and s["Enabled"]
]

if enabled_preview_list:
    print(f"\n⚠️  Enabled Preview Features ({len(enabled_preview_list)}):")
    for setting in enabled_preview_list[:10]:  # Show first 10
        print(f"  • {setting['Title']}")
        print(f"    ({setting['SettingName']})")
    
    if len(enabled_preview_list) > 10:
        print(f"  ... and {len(enabled_preview_list) - 10} more")
else:
    print("\n✓ No preview features are currently enabled")

print("\n" + "="*60)
print("✓ Monitoring completed successfully")
print("="*60)

## Query Log Analytics (Optional)

Example KQL queries to analyze the ingested data in Log Analytics.

```kql
// Preview features enabled in last 24 hours
FabricTenantSettings_CL
| where TimeGenerated > ago(24h)
| where IsPreviewFeature_b == true and Enabled_b == true
| project TimeGenerated, Title_s, SettingName_s, TenantSettingGroup_s
| order by TimeGenerated desc

// Settings change detection (compare with previous snapshot)
FabricTenantSettings_CL
| where TimeGenerated > ago(7d)
| summarize 
    Current = arg_max(TimeGenerated, Enabled_b),
    Previous = arg_min(TimeGenerated, Enabled_b)
    by SettingName_s
| where Current != Previous
| project SettingName_s, PreviousState=Previous, CurrentState=Current

// Preview features trend over time
FabricTenantSettings_CL
| where IsPreviewFeature_b == true
| summarize EnabledCount = countif(Enabled_b == true) by bin(TimeGenerated, 1d)
| render timechart
```